<a href="https://colab.research.google.com/github/talhaanwarch/Sarcasm-nlp/blob/master/twitter_multi_input_bert_preprocessed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
!pip install tensorflow==2.1.0
!pip install sentencepiece
!pip install tensorflow-hub
!pip install tensorflow-addons
#!wget --quiet https://raw.githubusercontent.com/google-research/ALBERT/master/tokenization.py

In [3]:
cd /content/drive/My\ Drive/dataset/sarcasm

/content/drive/My Drive/dataset/sarcasm


In [4]:
ls

model_gru.png
__pycache__/
sarcasm_detection_shared_task_reddit_testing.jsonl
sarcasm_detection_shared_task_reddit_training.jsonl
sarcasm_detection_shared_task_twitter_testing.jsonl
sarcasm_detection_shared_task_twitter_training.jsonl
tokenization.py


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import re
import string
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
twitter_train=pd.read_json('sarcasm_detection_shared_task_twitter_training.jsonl',lines=True)

In [7]:
twitter_train.head()

,label,response,context
0,SARCASM,@USER @USER @USER I don't get this .. obviousl...,[A minor child deserves privacy and should be ...
1,SARCASM,@USER @USER trying to protest about . Talking ...,[@USER @USER Why is he a loser ? He's just a P...
2,SARCASM,@USER @USER @USER He makes an insane about of ...,[Donald J . Trump is guilty as charged . The e...
3,SARCASM,@USER @USER Meanwhile Trump won't even release...,[Jamie Raskin tanked Doug Collins . Collins lo...
4,SARCASM,@USER @USER Pretty Sure the Anti-Lincoln Crowd...,[Man ... y ’ all gone “ both sides ” the apoca...


In [0]:
twitter_train['response']=twitter_train['response'].str.replace('@USER', "") 
twitter_train['response']=twitter_train['response'].str.replace('\d+', '')
twitter_train['response']=twitter_train['response'].str.lower()
twitter_train['response']=twitter_train['response'].str.replace('[^\w\s]','')


In [9]:
twitter_train.head()

,label,response,context
0,SARCASM,i dont get this obviously you do care or y...,[A minor child deserves privacy and should be ...
1,SARCASM,trying to protest about talking about him a...,[@USER @USER Why is he a loser ? He's just a P...
2,SARCASM,he makes an insane about of money from the ...,[Donald J . Trump is guilty as charged . The e...
3,SARCASM,meanwhile trump wont even release his sat sc...,[Jamie Raskin tanked Doug Collins . Collins lo...
4,SARCASM,pretty sure the antilincoln crowd claimed th...,[Man ... y ’ all gone “ both sides ” the apoca...


In [0]:
twitter_train['context']=twitter_train['context'].apply(lambda x: ','.join(map(str, x)))
twitter_train['context']=twitter_train['context'].str.replace('@USER', "") 
twitter_train['context']=twitter_train['context'].str.lower()
twitter_train['context']=twitter_train['context'].str.replace('[^\w\s]','')
twitter_train['context']=twitter_train['context'].str.replace('\d+', '')


In [11]:
twitter_train.head()

,label,response,context
0,SARCASM,i dont get this obviously you do care or y...,a minor child deserves privacy and should be k...
1,SARCASM,trying to protest about talking about him a...,why is he a loser hes just a press secretar...
2,SARCASM,he makes an insane about of money from the ...,donald j trump is guilty as charged the evid...
3,SARCASM,meanwhile trump wont even release his sat sc...,jamie raskin tanked doug collins collins look...
4,SARCASM,pretty sure the antilincoln crowd claimed th...,man y all gone both sides the apocalypse o...


In [12]:
len(twitter_train)

5000

In [13]:
from collections import Counter
Counter(twitter_train['label'])

Counter({'NOT_SARCASM': 2500, 'SARCASM': 2500})

In [0]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
train_label=le.fit_transform(twitter_train['label'])

In [15]:
twitter_test=pd.read_json('sarcasm_detection_shared_task_twitter_testing.jsonl',lines=True)
twitter_test.head()

,context,response,id
0,"[Well now that ’ s problematic AF <URL>, @USER...","@USER @USER @USER My 3 year old , that just fi...",twitter_1
1,[Last week the Fake News said that a section o...,@USER @USER How many verifiable lies has he to...,twitter_2
2,[@USER Let ’ s Aplaud Brett When he deserves i...,@USER @USER @USER Maybe Docs just a scrub of a...,twitter_3
3,[Women generally hate this president . What's ...,@USER @USER is just a cover up for the real ha...,twitter_4
4,"[Dear media Remoaners , you excitedly sharing ...",@USER @USER @USER The irony being that he even...,twitter_5


In [0]:
twitter_test['response']=twitter_test['response'].str.replace('@USER', "") 
twitter_test['response']=twitter_test['response'].str.replace('\d+', '')
twitter_test['response']=twitter_test['response'].str.lower()
twitter_test['response']=twitter_test['response'].str.replace('[^\w\s]','')
twitter_test['context']=twitter_test['context'].apply(lambda x: ','.join(map(str, x)))
twitter_test['context']=twitter_test['context'].str.replace('@USER', "") 
twitter_test['context']=twitter_test['context'].str.lower()
twitter_test['context']=twitter_test['context'].str.replace('[^\w\s]','')
twitter_test['context']=twitter_test['context'].str.replace('\d+', '')


In [17]:
twitter_test.head()

,context,response,id
0,well now that s problematic af url my year ...,my year old that just finished reading ni...,twitter_1
1,last week the fake news said that a section of...,how many verifiable lies has he told now d...,twitter_2
2,let s aplaud brett when he deserves it he co...,maybe docs just a scrub of a coach i mean ...,twitter_3
3,women generally hate this president whats up ...,is just a cover up for the real hate inside ...,twitter_4
4,dear media remoaners you excitedly sharing cl...,the irony being that he even has to ask why,twitter_5


#Preprocessing

In [0]:
# Thanks to https://www.kaggle.com/rftexas/text-only-kfold-bert
def clean_tweets(tweet):
    """Removes links and non-ASCII characters"""
    
    tweet = ''.join([x for x in tweet if x in string.printable])
    
    # Removing URLs
    tweet = re.sub(r"http\S+", "", tweet)
    
    return tweet

In [0]:
# Thanks to https://www.kaggle.com/rftexas/text-only-kfold-bert
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [0]:
def remove_punctuations(text):
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`"
    
    for p in punctuations:
        text = text.replace(p, f' {p} ')

    text = text.replace('...', ' ... ')
    
    if '...' not in text:
        text = text.replace('..', ' ... ')
    
    return text

In [0]:
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

In [0]:
# Thanks to https://www.kaggle.com/rftexas/text-only-kfold-bert
def convert_abbrev(word):
    return abbreviations[word.lower()] if word.lower() in abbreviations.keys() else word

In [0]:
def convert_abbrev_in_text(text):
    tokens = word_tokenize(text)
    tokens = [convert_abbrev(word) for word in tokens]
    text = ' '.join(tokens)
    return text

In [24]:
twitter_train.head()

,label,response,context
0,SARCASM,i dont get this obviously you do care or y...,a minor child deserves privacy and should be k...
1,SARCASM,trying to protest about talking about him a...,why is he a loser hes just a press secretar...
2,SARCASM,he makes an insane about of money from the ...,donald j trump is guilty as charged the evid...
3,SARCASM,meanwhile trump wont even release his sat sc...,jamie raskin tanked doug collins collins look...
4,SARCASM,pretty sure the antilincoln crowd claimed th...,man y all gone both sides the apocalypse o...


In [0]:
twitter_train["response"] = twitter_train["response"].apply(lambda x: clean_tweets(x))
twitter_train["context"] = twitter_train["context"].apply(lambda x: clean_tweets(x))

twitter_train["response"] = twitter_train["response"].apply(lambda x: remove_emoji(x))
twitter_train["context"] = twitter_train["context"].apply(lambda x: remove_emoji(x))

twitter_train["response"] = twitter_train["response"].apply(lambda x: remove_punctuations(x))
twitter_train["context"] = twitter_train["context"].apply(lambda x: remove_punctuations(x))

twitter_train["response"] = twitter_train["response"].apply(lambda x: convert_abbrev_in_text(x))
twitter_train["context"] = twitter_train["context"].apply(lambda x: convert_abbrev_in_text(x))

In [0]:
twitter_test["response"] = twitter_test["response"].apply(lambda x: clean_tweets(x))
twitter_test["context"] = twitter_test["context"].apply(lambda x: clean_tweets(x))

twitter_test["response"] = twitter_test["response"].apply(lambda x: remove_emoji(x))
twitter_test["context"] = twitter_test["context"].apply(lambda x: remove_emoji(x))

twitter_test["response"] = twitter_test["response"].apply(lambda x: remove_punctuations(x))
twitter_test["context"] = twitter_test["context"].apply(lambda x: remove_punctuations(x))

twitter_test["response"] = twitter_test["response"].apply(lambda x: convert_abbrev_in_text(x))
twitter_test["context"] = twitter_test["context"].apply(lambda x: convert_abbrev_in_text(x))

#BERT

In [0]:
import tensorflow_hub as hub
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [0]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [0]:
import tokenization
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [0]:
 max_len=256

In [0]:
train_response = bert_encode(twitter_train.response, tokenizer, max_len=max_len)
train_context = bert_encode(twitter_train.context, tokenizer, max_len=max_len)
train_labels=twitter_train.label.values
#test_input = bert_encode(test_bert.values, tokenizer, max_len=160)

In [0]:
test_response = bert_encode(twitter_test.response, tokenizer, max_len=max_len)
test_context = bert_encode(twitter_test.context, tokenizer, max_len=max_len)

In [0]:
test_gen=[test_context[0],test_context[1],test_context[2],test_response[0],test_response[1],test_response[2]]

In [0]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,concatenate,Dense,GlobalAveragePooling1D,Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import tensorflow_addons as tfa

In [0]:
contex_input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="contex_input_word_ids")
contex_input_mask = Input(shape=(max_len,), dtype=tf.int32, name="contex_input_mask")
contex_segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="contex_segment_ids")
_, context_sequence_output = bert_layer([contex_input_word_ids, contex_input_mask, contex_segment_ids])
context_clf_output = context_sequence_output[:, 0, :]


reponse_input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="reponse_input_word_ids")
reponse_input_mask = Input(shape=(max_len,), dtype=tf.int32, name="reponse_input_mask")
reponse_segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="reponse_segment_ids")
_, reponset_sequence_output = bert_layer([reponse_input_word_ids, reponse_input_mask, reponse_segment_ids])
reponset_clf_output = reponset_sequence_output[:, 0, :]



conc=concatenate([context_clf_output,reponset_clf_output])
#conc=Dropout(0.3)(conc)
out = Dense(1, activation='sigmoid')(conc)
    
model = Model(inputs=[contex_input_word_ids, contex_input_mask, contex_segment_ids,reponse_input_word_ids,reponse_input_mask,reponse_segment_ids], outputs=out)
model.compile(Adam(1e-6), loss='binary_crossentropy', metrics=['accuracy'])
    


In [0]:
train_gen=[train_context[0],train_context[1],train_context[2],train_response[0],train_response[1],train_response[2]]

In [0]:
train_history = model.fit(
    train_gen, train_label,
    epochs=3,
    batch_size=6,
    validation_split=0.1)

Train on 4500 samples, validate on 500 samples
Epoch 1/3


In [0]:
# len(train_gen),len(train_label)

In [0]:
# train_context[0].shape

In [0]:
# import tensorflow_addons as tfa
# from sklearn.model_selection import StratifiedKFold


In [0]:
# def unpack(a, b, c, d,e,f,g): 
#     #print(a[g].shape, b[g].shape, c[g].shape, d[g].shape,e[g].shape,f[g].shape) 
#     return [a[g], b[g], c[g], d[g],e[g],f[g]]

In [0]:
#unpack(*train_gen,train)

In [0]:
# from sklearn.model_selection import StratifiedKFold
# kfold = StratifiedKFold(n_splits=5, shuffle=False)
# scores = []
# test=[]
# s=0
# for train, val in kfold.split(list(zip(*train_gen)), train_label):
#   model.fit(unpack(*train_gen,train),train_label[train],batch_size=6,epochs=5,verbose=2,
#             validation_data=(unpack(*train_gen,val), train_label[val]))
   

#   y_pred = model.predict(unpack(*train_gen,val), batch_size=6, verbose=1)

#   test.append(model.predict(test_gen, batch_size=6, verbose=1).ravel())

#   y_pred = (y_pred > 0.5)
#   f1=f1_score(train_label[val], y_pred, average='macro')
#   s+=1
#   print(s,' f1 score  is ',f1)
#   scores.append(f1)
#   tf.keras.backend.clear_session()

In [0]:
test=model.predict(test_gen, batch_size=6, verbose=1).ravel()

In [0]:
test.shape

In [0]:

sub=test.round()
sub=le.inverse_transform(sub.ravel().astype('int16'))
sub=pd.DataFrame(sub,columns=['label'])
sub=pd.concat([twitter_test['id'], sub], axis=1)
sub.head()
sub.to_csv('/content/answer.txt',sep=',',index=False,header=None)

In [0]:
test[0].shape